In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from gensim import models
import sys
sys.path.insert(0, "/home/luke/Github/Project/python")
import compute_healthiness as ch
import clean_recipes_datasets as clean

### load data

In [2]:
embeddings_path = "../generated/food_embeddings.bin"
food_embeddings = models.Word2Vec.load(embeddings_path)

# Table of conversion for the unit found in 1M
with open('../generated/convert_gr.json') as json_data:
    convert_gr = json.load(json_data)
    
# Mapping from ingredient to USDA id
with open('../generated/ing_id_mapping.json') as json_data:
    mapping_usda_id = json.load(json_data)
    
#nutrients table
nutrients_mapping = pd.read_hdf("../generated/nut_data.h5", 'table')

id_repr = json.load(open("../generated/id_repr.json"))
repr_ids = json.load(open("../generated/repr_ids.json"))
unit_quantities = json.load(open("../generated/1m_unit_quantities.json"))

### define example recipes

In [18]:
recipe1 = [[1.0, 'c.', 'elbow macaroni'],
  [1.0, 'c.', 'American cheese'],
  [0.5, 'c.', 'celery'],
  [0.5, 'c.', 'green peppers'],
  [3.0, 'tbsp', 'pimentos'],
  [0.5, 'c.', 'mayonnaise'],
  [1.0, 'tbsp', 'vinegar'],
  [0.75, 'teaspoon', 'salt'],
  [0.5, 'teaspoon', 'dry dill weed']]

recipe2 = [[12.0, 'cup', 'shredded coconut'],
  [1.0, 'lb', 'lean ground beef'],
  [1.5, 'tablespoon', 'fresh garlic'],
  [2.5, 'g', 'kosher salt'],
  [1.0, 'tablespoon', 'lemon juice'],
  [1.0, 'tablespoon', 'soy sauce'],
  [2.0, 'tablespoon', 'cornstarch'],
  [8.0, 'ounce', 'pineapple chunks'],
  [16.0, 'ounce', 'mandarin oranges'],
  [12.0, 'cup', 'cashew nuts']]

recipe3 = [[1.0, 'lb', 'lean ground beef'],
    [1.0, '', 'onion'],
    [1, 'tsp', 'chili powder'],
    [0.5, 'tsp', 'salt'],
    [0.5, 'tsp', 'garlic powder'],
    [8, 'oz', 'tomato sauce'],
    [10, 'oz', 'taco shells'],
    [6, 'oz', 'cheddar cheese'],
    [2, 'c.', 'lettuce'],
    [2, '', 'tomatoes'],
    [0.75, 'c.', 'salsa'],
    [0.75, 'c.', 'sour cream']]

### design recommender

We first estmate the healthiness score for a given recipe

In [4]:
input_recipe = recipe2
ch.compute_healthiness(input_recipe, mapping_usda_id, nutrients_mapping, convert_gr, unit_quantities)

shredded coconut    : 35.90% =>  content(grams): (fat=346.3680, sat_fat=308.7206, sugar=693.6576, salt=3.7018, energy=7065.6000)
lean ground beef    : 10.60% =>  content(grams): (fat=12.2016, sat_fat= 4.6811, sugar= 0.0000, salt=0.2495, energy=530.7026)
fresh garlic        :  0.50% =>  content(grams): (fat= 0.1073, sat_fat= 0.0191, sugar= 0.2145, salt=0.0036, energy=31.9605)
kosher salt         :  0.06% =>  content(grams): (fat= 0.0000, sat_fat= 0.0000, sugar= 0.0000, salt=0.9690, energy=0.0000)
lemon juice         :  0.33% =>  content(grams): (fat= 0.0343, sat_fat= 0.0057, sugar= 0.3604, salt=0.0001, energy=3.1460)
soy sauce           :  0.33% =>  content(grams): (fat= 0.0143, sat_fat= 0.0016, sugar= 0.2431, salt=0.7988, energy=8.5800)
cornstarch          :  0.67% =>  content(grams): (fat= 0.0143, sat_fat= 0.0026, sugar= 0.0000, salt=0.0026, energy=108.9660)
pineapple chunks    :  5.24% =>  content(grams): (fat= 0.2240, sat_fat= 0.0157, sugar=47.2640, salt=0.0045, energy=192.6400)
man

{'fat': 35.10995253309952,
 'sat_fat': 57.20625283038799,
 'sugar': 22.567201387697597,
 'salt': 1.384310411798608,
 'energy': 23.864062974584588,
 'total': 140.1317801375683}

We now design a strategy for finding replacements

In [5]:
#the bigger the better
def health_similarity_score(similarity, health) :
    if health <= 7 :
        return similarity
    else :
        return (similarity**2 / health) 
        

def find_swaps(ing_id, threshold=0.45, nb=1) : 
    
    try :        
        ing_ref = id_repr[str(ing_id)] 
        
    except KeyError:
        print("The ingredient with id", ing_id, "has never been seen before")  
        return None
    
    closest_refs = food_embeddings.wv.most_similar(ing_ref, topn=50)
    
    to_consider = []
    
    closest_refs_and_contains = list(filter(lambda x : (ing_ref     in x[0]) and (x[1] >= threshold), closest_refs))
    closest_refs_not_contains = list(filter(lambda x : (ing_ref not in x[0]) and (x[1] >= threshold), closest_refs))
    
    if len(closest_refs_not_contains) == 0 :
        to_consider = closest_refs_and_contains
    else :
        to_consider = closest_refs_not_contains
        
    if len(to_consider) == 0 :
        return None
    
    else :
        entries = []
        fat, sat_fat, sugar, salt, energy = ch.compute_profile([(100.0, ing_id)], nutrients_mapping)
        base_score = ch.score(fat, sat_fat, sugar, salt, energy)['total']
        for ref in to_consider :
            
            best_id = -1
            best_score = np.inf
            
            for collided_id in repr_ids[ref[0]] :
                fat, sat_fat, sugar, salt, energy = ch.compute_profile([(100.0, collided_id)], nutrients_mapping)
                score = ch.score(fat, sat_fat, sugar, salt, energy)['total']

                if score < best_score :
                    best_id = collided_id
                    best_score = score
            
            
            if (score < base_score) :
                entries.append((ref[0], ref[1], score))
        if len(entries) == 0 :
            return None
        else:
            return sorted(entries, key=lambda x : health_similarity_score(x[1], x[2]), reverse=True)[:nb]

In [12]:
def find_swapping(recipe) :
    swappings = []
    
    recipe_info = ch.map_one_recipe_usda(recipe, mapping_usda_id, convert_gr, unit_quantities)
    if recipe_info is None :
        print("No swapping found")
    
    best_score = -1
    best_swap = (None, None) 
    
    for e, (quant, ingr) in enumerate(recipe_info) :
        print
        swap = find_swaps(ingr)

        if swap is not None :
            swap = swap[0]
            recipe_copy = [i for i in recipe_info]
            recipe_copy[e] = (recipe_copy[e][0],mapping_usda_id[swap[0]])
            
            fat, sat_fat, sugar, salt, energy = ch.compute_profile(recipe_copy, nutrients_mapping)
            score = ch.score(fat, sat_fat, sugar, salt, energy)['total']
            score = health_similarity_score(swap[1], score)
            
            if score > best_score :
                best_score = score
                best_swap = (recipe[e][2], swap[0])
                
    return best_swap

def find_consecutive_swappings(recipe, nb_swaps=3) :
    recipe_copy = [ing.copy() for ing in recipe]
    swaps = []
    
    for swap_iter in range(nb_swaps) :
        current_swap = find_swapping(recipe)
        swaps.append(current_swap)
        
        #replace ingredient in recipe
        for e, ing in enumerate(recipe) :
            if ing[2] == current_swap[0] :
                recipe[e][2] = current_swap[1]
                
    return swaps, recipe

In [20]:
swaps, recipe = find_consecutive_swappings(recipe1, nb_swaps=100)
print(swaps)
print()
print(recipe)

[('artichoke', 'asparagu'), ('asparagu', 'fiddlehead'), ('tuna in water', 'surimi'), ('surimi', 'alfalfa'), ('alfalfa', 'green lettuce'), ('green lettuce', 'lettuce'), ('green peppers', 'kidney bean'), ('kidney bean', 'tomato'), ('tomato', 'yellow pepper'), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), 

In [7]:
find_swaps(mapping_usda_id["egg yolk"], nb=10)

/home/luke/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('raspberry puree', 0.5070383548736572, 6.920142857142857),
 ('gooseberry', 0.4925079643726349, 1.326),
 ('raspberry', 0.4789777994155884, 4.13),
 ('boysenberry', 0.4786318242549896, 6.414),
 ('cherry pie filling', 0.4575006067752838, 2.011),
 ('espresso', 0.450120210647583, 0.639),
 ('cream of tartar', 0.6337701082229614, 9.899999999999999),
 ('sour cherry', 0.46027106046676636, 8.603428571428573),
 ('black cherry', 0.49835866689682007, 14.118285714285713),
 ('pudding rice', 0.5060806274414062, 18.440571428571428)]

In [11]:
find_swapping(recipe1)

('elbow macaroni', 'pasta')

In [10]:
marmiton_top1 = [[1000, "g", "veal"], 
                 [1, "tsp", "chicken broth"], 
                 [2, "", "carrots"], 
                 [1, "", "onion"],
                [1, "cup", "mushrooms"], 
                [1, "cup", "cream"],
                [1, "", "lemon"],
                [1, "", "egg yolk"],
                [1, "cup", "flour"],
                [250, "g", "white wine"],
                [2.5, "g", "kosher salt"],
                [2.5, "g", "pepper"]]

best_swap = find_swapping(marmiton_top1)
print("We suggest to replace " + best_swap[0] + " by " + best_swap[1])

We suggest to replace veal by rabbit
